# **Tools Recognition**

In [ ]:
# Istallazione pacchetti se necessario.

# pip install opencv-python

In [1]:
# pacchetti utilizzati.
import os
import cv2
import glob
from PIL import Image
from tqdm import tqdm

### 1) Raccolta e etichettatura delle immagini

- 10 diverse categorie:
    - accendino
    - cacciavite
    - chiave
    - forbici
    - martello
    - metro
    - nastro
    - pappagallo
    - penna
    - spillatrice 

- 4 diverse superfici:
    - tavolo bianco
    - banco da lavoro
    - pavimento di cemento
    - coperta a righe
- 20 immagini per il training per ogni categoria
- 26 immagini per il test contenenti oggetti multipli e anche estranei alle 10 categorie


In [2]:
# Oggetti utilizzati.
tool_names = ['accendino', 'cacciavite', 'chiave', 'forbici', 'martello', 'metro', 'nastro', 'pappagallo', 'penna', 'spillatrice']

In [27]:
# Rinomiamo le immagini nelle cartelle.
cwd = os.getcwd()
for tool in tool_names:
    path = os.path.join(cwd, f"images/{tool}/")
    image_files = glob.glob(path + '*.png')

    for i, file in enumerate(image_files):
        new_name = tool+ '_' + str(i+1) + '.png'
        os.rename(file, os.path.join(path, new_name))


# Rinominiamo le immagini del test set.
path = os.path.join(cwd, f"images/Test_set/")
image_files = glob.glob(path + '*.png')

for i, file in enumerate(image_files):
    new_name = 'test_' + str(i+1) + '.png'
    os.rename(file, os.path.join(path, new_name))

### 2) Pre-elaborazione delle immagini

- Ridimensionamento : portartiamo nello stesso formato tutte le immagini (520 x 520). Alcune delle utilità di questo passaggio possono essere una maggiore uniformità una volta che si va ad allenare l'algoritmo di calssificazione e l'aumento della diversità dei dati di training che possono portare il modello ad avere una maggiore robustezza.

In [ ]:
# Portiamo tutte le immagini nella stessa dimensione (520 x 520).
cwd = os.getcwd()
for tool in tool_names:
    path = os.path.join(cwd, f"images/{tool}/")
    image_files = glob.glob(path + '*.png')

    for i, file in enumerate(image_files):
        image = Image.open(file)

        # Ridimensionamento dell'immagine.
        resized_image = image.resize((520, 520))
        resized_image.save(file)

- Riduzione del rumore:  è finalizzata a migliorare la qualità dell'immagine, eliminando il rumore e aumentando il contrasto tra gli oggetti dell'immagine e lo sfondo. 

In [ ]:
# Riduzione del rumore.
cwd = os.getcwd()

for tool in tool_names:
    folder_path = os.path.join(cwd, f"images/{tool}/")

    for i in range(20):
        path = f'{folder_path}/{tool}_{i+1}.png'
        img = cv2.imread(path)

        # Applicazione del filtro di riduzione del rumore.
        img = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)
        new_path = f'{folder_path}/{tool}_{i+1}.1.png'
        cv2.imwrite(new_path, img)

- Eliminazione delle ombre: eleminando le ombre è possibile utilizzare successivamente delle tecniche di segmentazione basate sui contorni essendo le forme degli utensili più delineate e non deformate dalle ombre.

In [ ]:
cwd = os.getcwd()

for tool in tqdm(tool_names):
    folder_path = os.path.join(cwd, f"images/{tool}/")

    for i in range(20):
        path = f'{folder_path}/{tool}_{i+1}.png'
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

        # Applica soglia adattiva.
        thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

        # Rimuove le ombre mascherando l'immagine con la soglia.
        removed_shadows_img = cv2.bitwise_and(img, thresh)
        new_path = f'{folder_path}/{tool}_{i+1}.1.png'
        cv2.imwrite(new_path, removed_shadows_img)

### 3) Segmentazione delle immagini 

In [56]:
new_path

'c:\\Users\\mcm23\\OneDrive\\Desktop\\GitHub\\Object_Detection\\images/accendino/accendino_2.2.png'

In [71]:
new_path = 'c:\\Users\\mcm23\\OneDrive\\Desktop\\GitHub\\Object_Detection\\images/metro/metro_16.1.png'
img = cv2.imread(new_path, cv2.IMREAD_GRAYSCALE)
# Applica la sogliatura di Otsu
_, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Visualizza l'immagine segmentata
cv2.imshow('Thresholded Image', thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
a = 'accendino'

new_path = os.path.join(cwd, f"images/{a}/{a}_2.2.png")
img = cv2.imread(new_path, cv2.IMREAD_GRAYSCALE)

# Applica la sogliatura di Otsu
_, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Visualizza l'immagine segmentata
cv2.imshow('Thresholded Image', thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Eventuale ricolorazione in scala di grigi.
cwd = os.getcwd()

for tool in tqdm(tool_names):
    folder_path = os.path.join(cwd, f"images/{tool}/")

    for i in range(20):
        path = f'{folder_path}/{tool}_{i+1}.png'
        # Caricamento dell'immagine.
        img = Image.open(path)

        # Conversione in scala di grigi
        img = img.convert('L')

        # Salvataggio nuova immagine.
        new_path = f'{folder_path}/{tool}_{i+1}.1.png'
        img.save(new_path)




# Riduzione delle ombre.

# Load the image
img = cv2.imread(new_path, cv2.IMREAD_GRAYSCALE)

# Apply adaptive thresholding
thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

# Remove shadows by masking the thresholded image
removed_shadows = cv2.bitwise_and(img, thresh)
cv2.imshow('Thresholded Image', removed_shadows)
cv2.waitKey(0)
cv2.destroyAllWindows()

Ecco una possibile pipeline di elaborazione per un progetto di object detection utilizzando un modello di classificazione e un metodo di segmentazione:

1) ``Raccolta dei dati``: Prima di iniziare qualsiasi progetto di object detection, è necessario raccogliere un dataset di immagini etichettate con le classi di oggetti di interesse. Il dataset deve essere rappresentativo delle varie condizioni di luce, sfondo e posizione degli oggetti.

2) ``Pre-elaborazione dell'immagine``: L'immagine deve essere preparata per il successivo passaggio di segmentazione. Si può effettuare la riduzione del rumore o il bilanciamento del colore.

3) ``Segmentazione dell'immagine``: Una volta ottenuto l'immagine preparata, si applica un metodo di segmentazione per dividere l'immagine in regioni omogenee. Si possono utilizzare tecniche di segmentazione come la segmentazione basata sulla soglia, la segmentazione basata sulla regione o la segmentazione basata sui contorni.

4) ``Estrazione delle features``: Per ogni regione dell'immagine segmentata, si estraggono le feature utilizzate per addestrare il modello di classificazione. Le feature possono includere la forma, la texture, il colore o qualsiasi altra caratteristica che aiuti a distinguere le diverse classi di oggetti.

5) ``Addestramento del modello di classificazione``: Una volta estratte le feature, si addestra il modello di classificazione utilizzando un algoritmo di machine learning come le reti neurali, le SVM o gli alberi di decisione. Il modello viene addestrato per riconoscere le diverse classi di oggetti.

6) ``Rilevamento degli oggetti``: Una volta addestrato il modello di classificazione, si può applicare il modello all'immagine originale per rilevare gli oggetti di interesse. Utilizzando le feature estratte, il modello cerca le regioni dell'immagine che corrispondono agli oggetti di interesse.

7) ``Post-elaborazione dell'immagine``: Una volta rilevati gli oggetti, si possono applicare tecniche di post-elaborazione come la non-maximum suppression per rimuovere i falsi positivi e migliorare la precisione del modello.

In sintesi, la pipeline di elaborazione per un progetto di object detection con un modello di classificazione e un metodo di segmentazione consiste in: raccolta dei dati, pre-elaborazione dell'immagine, segmentazione dell'immagine, estrazione delle feature, addestramento del modello di classificazione, rilevamento degli oggetti e post-elaborazione dell'immagine.